In [182]:
from LandBase import *

# 주피터노트북 매직커맨드 혹은 매직키워드
# 구글에서 찾을 때는 "jupyter notebook Built-in magic commands"라고 찾으면 수월함
# %lsmagic : 사용 가능한 매직커맨드 목록
# %quickref : 사용 가능한 매직커맨드 목록과 설명 (quick reference)
# %prun : 주피터노트북에서 실행된 내용의 퍼포먼스 측정
# %time : 주피터노트북에서 실행된 내용의 시간 측정
# %pdb : 디버깅 목적으로 오류가 발생하면 그 위치에서 멈추게 함
# %load_ext : 외부 플러그인 가져오기

def apt_sell_info(p_ym='201510') :
    ####
    # 분양정보목록
    # 출처: http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=년-월
    ####
    
    r_list = []
    url = 'http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=' + p_ym[:4] + '-' + p_ym[4:]
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml') # bs4.BeautifulSoup 타입
    # bs4.BeautifulSoup 타입에서 find 혹은 find_all만 한 결과는 bs4.element.ResultSet 타입이다.
    # soup.find_all('table', 'esale_cal_table3') = soup.find_all('table', class_='esale_cal_table3')
    # soup.find_all('table', class_='esale_cal_table3') : table 태그 중에서 class가 esale_cal_table3인 경우를 모두 찾음, bs4.element.ResultSet 타입
    # soup.find('table', 'esale_cal_table3').find_all('tr') : 위에서 찾은 결과에서 tr 태그를 가진 값 모두 찾음, bs4.element.ResultSet 타입
    for tr in soup.find('table', class_='esale_cal_table3').find_all('tr') :
        # tr.find('th').find('span', 'td_date').find('strong') : 위에 찾은 결과에서 th 태그 -> span태그 & class가 td_date이고 -> strong 태그
        # tr.find('th').find('span', 'td_date').find('strong').string : 위 결과에서 문자열로 된 값만 추출, bs4.element.NavigableString 타입, 여기서는 필요없으므로 안 찾음
        for li in tr.find_all('li', class_=False) :
            분양유형 = '민간분양'
            형태 = li.find('span').string.strip('[]')
            if 형태 == '오피스텔' : 
                continue # 오피스텔은 건너뛰기
            apt_nm = li.find('a')['title'].strip()
            if not apt_nm.startswith('서울') and not apt_nm.startswith('인천') and not apt_nm.startswith('경기') :
                continue # 서울, 인천, 경기 아니면 건너뛰기
            apt_no = re.findall('\d+', li.find('a')['href'])[0]
            if apt_nm.count('뉴스테이') :
                분양유형 = '뉴스테이'
                apt_nm = apt_nm.replace('뉴스테이', '')
            elif apt_nm.count('공공임대') :
                분양유형 = '공공임대'
                apt_nm = apt_nm.replace('공공임대', '')
            elif apt_nm.count('영구임대') :
                분양유형 = '영구임대'
                apt_nm = apt_nm.replace('영구임대', '')
            elif apt_nm.count('국민임대') :
                분양유형 = '국민임대'
                apt_nm = apt_nm.replace('국민임대', '')
            elif apt_nm.count('공공분양') :
                분양유형 = '공공분양'
                apt_nm = apt_nm.replace('공공분양', '')
            if apt_nm.endswith('순위') :
                apt_nm = apt_nm.rsplit(' ', 1)[0] # 오른쪽부터 1번만 분리
            apt_nm = apt_nm.replace('()', '').replace(',)', ')').replace(', )', ')')
            
            r_dict = apt_basic_info(apt_no) # 단지 기본정보 함수 호출
            if '입주시기' not in r_dict.keys() :
                r_dict['입주시기'] = str(int(p_ym[:4])+3) + p_ym[4:]
            if '특징' not in r_dict.keys() :
                r_dict['특징'] = ''
            
            r_list.append([p_ym, r_dict['입주시기'], 분양유형, 형태, apt_no, apt_nm, r_dict['주소'], r_dict['동수'], r_dict['가구수'], r_dict['최고층'], r_dict['특징']])
    
    df = pd.DataFrame(r_list, columns=['sell_ym', 'move_ym', 'sell_type', 'apt_type', 'apt_no', 'apt_nm', 'address', 'building_cnt', 'house_cnt', 'top_floor', 'characteristics'])
    df.drop_duplicates(inplace=True)
    return df
    
def apt_basic_info(p_no='39801') :
    ####
    # 단지기본 정보
    # 출처: http://www.drapt.com/sise/html_view5/total.php?no=단지번호
    # 정보: 최고층, 동수, 가구수, 입주시기, 주소, 특징
    ####
    
    r_dict = {'no':p_no}
    url = 'http://www.drapt.com/sise/html_view5/total.php?no=' + p_no

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml') # bs4.BeautifulSoup 타입
    
    for tr in soup.find('table').find_all('tr') :
        th = tr.find_all('th')
        td = tr.find_all('td')
        for i in range(len(th)) :
            th_str = th[i].string.replace('\n', '').strip()
            td_str = td[i].contents[0].string.replace('\n', '').strip()
            if th_str == '단지규모' :
                r_dict['최고층'] = td_str.split()[0].split('~')[-1].replace('층', '')
                r_dict['동수'] = td_str.split()[1].replace('개동', '')
                r_dict['가구수'] = td_str.split()[-1].replace('총', '').replace('가구', '')
            elif th_str == '입주시기' :
                r_dict[th_str] = td_str.replace('년', '').replace('월', '').replace(' ', '')[:6]
            elif th_str in ['주소', '특징'] :
                r_dict[th_str] = td_str
    
    #display(r_dict)
    return r_dict

def apt_size_info(p_ym='201510', p_no='53685') :
    ####
    # 단지면적 정보
    # 출처: http://www.drapt.com/sise/html_view5/size_info.php?no=53685
    # 정보: 공급면적, 전용면적, 방/욕실, 가구수(임대)
    ####
    
    url = 'http://www.drapt.com/sise/html_view5/size_info.php?no=' + p_no

    fr_list = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml') # bs4.BeautifulSoup 타입
    
    for tr in soup.find('table').find_all('tr') :
        r_list = []
        td = tr.find_all('td')
        if len(td) == 0 :
            continue
        r_list.append(p_ym)
        r_list.append(p_no)
        r_list.append(td[0].find('a', class_='area_big').contents[0]) # 공급면적
        r_list.append(td[1].find('span').contents[0].replace('㎡','')) # 전용면적
        r_list.append(td[5].string.split('/')[0]) # 방
        r_list.append(td[5].string.split('/')[1]) # 욕실
        house_cnt = rent_house_cnt = td[6].find('span').contents[0]
        house_cnt = house_cnt.split('(')[0]
        if len(house_cnt) == 0 :
            house_cnt = 0
        else :
            house_cnt = int(house_cnt)
        if len(rent_house_cnt.split('(')) == 1 : # 임대가구수가 없는 경우
            rent_house_cnt = 0 # 임대가구수
        else:
            rent_house_cnt = int(rent_house_cnt.split('(')[1].replace(')','')) # 임대가구수
        r_list.append(house_cnt) # 일반가구수
        r_list.append(rent_house_cnt) # 임대가구수
        
        fr_list.append(r_list)
    
    df = pd.DataFrame(fr_list, columns=['sell_ym', 'no', 'supply_area', 'priv_area', 'room_cnt', 'bath_cnt', 'house_cnt', 'rent_house_cnt'])
    display(df)

In [183]:
# DB Connection Open
#con = get_db_connection()

# 닥터아파트에서 월별 분양정보 가져오기
ym_list = get_ym_list('201510', '201510')
for ym in ym_list :
    df = apt_sell_info(ym)
    for apt_no in df['apt_no'] :
        print(ym, apt_no)
        apt_size_info(ym, apt_no)

# DB Connection Close
#disconnect_db(con)

print('Program Finised')

201510 39801


,sell_ym,no,supply_area,priv_area,room_cnt,bath_cnt,house_cnt,rent_house_cnt
0,201510,39801,85.16,59,3,2,0,160
1,201510,39801,101.85,74,3,2,0,928
2,201510,39801,115.18A,84,3,2,0,364
3,201510,39801,114.89C,84,3,2,0,94
4,201510,39801,115.40D,84,3,2,0,108
5,201510,39801,115.73B,84,3,2,0,746


201510 53386


,sell_ym,no,supply_area,priv_area,room_cnt,bath_cnt,house_cnt,rent_house_cnt
0,201510,53386,34,21,,1,0,204
1,201510,53386,41,26,,1,0,198


201510 53685


...

201510 53717


,sell_ym,no,supply_area,priv_area,room_cnt,bath_cnt,house_cnt,rent_house_cnt
0,201510,53717,117.09A,84,3,2,546,0
1,201510,53717,117.60B,84,3,2,123,0
2,201510,53717,118.25C,84,3,2,219,0
3,201510,53717,131.03,95,3,2,275,0


201510 53356


,sell_ym,no,supply_area,priv_area,room_cnt,bath_cnt,house_cnt,rent_house_cnt
0,201510,53356,71.29A,51,2,1,409,0
1,201510,53356,71.26B,51,2,1,56,0
2,201510,53356,102.86A,74,3,2,200,0
3,201510,53356,116.46A,84,3,2,280,0
4,201510,53356,115.99B,84,3,2,80,0


Program Finised
